# Generative Model

## Naive Bayes classification

Asumption:

- h ~ Bernoulli(phi)
- D|h ~ Bernoulli(D)
- Input data is conditional independent given y, i.e $p(D_i|D_j,y) = p(D_i|y)\forall i,j$ 

With continous features problem, we can discretize a continous feature into ***k*** bounded ranges and then count them to find:

$P(D_i=d_{ir}| h = 0)$ and $P(D_i=d_{ir}| h = 1) \forall r \in k=\{<2,>=2 \&<5,...\}$


In [407]:
from math import pi, sqrt, exp, log, inf
from random import randrange, seed
from csv import reader
import numpy as np

## Create dataset

In [408]:
dataset = [[3.393533211, 2.331273381, 0],
           [3.110073483, 1.781539638, 0],
           [1.343808831, 3.368360954, 0],
           [3.582294042, 4.67917911, 0],
           [2.280362439, 2.866990263, 0],
           [7.423436942, 4.696522875, 1],
           [5.745051997, 3.533989803, 1],
           [9.172168622, 2.511101045, 1],
           [7.792783481, 3.424088941, 1],
           [7.939820817, 0.791637231, 1]]


## Find mean, variance, length of each attributes (columns), except for the last column (ground truth value)

In [409]:
def mean(X):
    return sum(X)/ len(X)

def stdev(X):
	mu = mean(X)
	return  sum((x - mu)**2 for x in X) / (len(X) - 1)

In [410]:
d = [[1, 3, 4],
     [1, 2, 3],
     [2, 3, 4],
     [1, 2, 3]]
print(*d)
for column in zip(*d):
    print(column)

[1, 3, 4] [1, 2, 3] [2, 3, 4] [1, 2, 3]
(1, 1, 2, 1)
(3, 2, 3, 2)
(4, 3, 4, 3)


In [411]:
def summarize_data(dataset):
	# except the last column is ground truth
	rm_last_col = [row[:-1] for row in dataset]
	return [(mean(col), stdev(col), len(col)) for col in zip(*rm_last_col)]
summarize_data(dataset)

[(5.178333386499999, 7.653989826170761, 10),
 (2.9984683241, 1.4848795625703213, 10)]

In [412]:
def separated_by_class(dataset):
	"""
	Separate the data set into classes
	Assume the final colulmn is ground truth of the class
	"""
	classes = dict()
	for row in dataset:
		if row[-1] not in classes:
			classes[row[-1]] = list()
		classes[row[-1]].append(row)
	return classes
separated_by_class(dataset)

{0: [[3.393533211, 2.331273381, 0],
  [3.110073483, 1.781539638, 0],
  [1.343808831, 3.368360954, 0],
  [3.582294042, 4.67917911, 0],
  [2.280362439, 2.866990263, 0]],
 1: [[7.423436942, 4.696522875, 1],
  [5.745051997, 3.533989803, 1],
  [9.172168622, 2.511101045, 1],
  [7.792783481, 3.424088941, 1],
  [7.939820817, 0.791637231, 1]]}

In [413]:
def summarize_by_class(dataset):
	separated = separated_by_class(dataset)
	sumaries = dict()
	for class_val, rows_class_val in separated.items(): # items() means (keys, values)
		sumaries[class_val] = summarize_data(rows_class_val)
	return sumaries
print(summarize_by_class(dataset))

{0: [(2.7420144012, 0.8585288681757653, 5), (3.0054686692, 1.2261788197598094, 5)], 1: [(7.6146523718, 1.5238227453753934, 5), (2.9914679790000003, 2.1146776839446155, 5)]}


## Gaussian Probability

In [414]:
def gauss(x, mean, stdev):
    assert stdev !=0
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return 1 / (sqrt(2 * pi) * stdev ) * exponent

def log_gausian(x, mean, stdev):
    return - log(stdev) - 0.5 * (x-mean)**2/stdev**2 # removed constant: -0.5 * log(2*pi)

def pro_smooth(Nh, N, p):
    """
    Smooth technique
    prob(X) = (Nh+mp) / (N + m)

    where, 
        Nh: # occurences of the type A in class h
        N: # of total types in class h
        m: equivalent sample size (heuristic choice)
        p: probability of the type A
    """
    m = 3 
    return (Nh+m*p)/(N + m)
# log(gauss(0.014400, 0.022630, 0.000188)) will get zero. 
# Instead, we use log_gaussian that we computed by hand first.
log_gausian(0.014400, 0.022630, 0.000188)

-949.6160989014708

## Class Probabilities

In [415]:
def MAP(dataset, input_row):
    """
    Maximum a Posteriori (MAP) 
    h = argmax_h P(h|D) = argmax_h P(D|h) * P(h)
    P(D|h) = P(D1|h) * P(D2|h) * ... * P(Dn|h), assuming that these D1,D2,... is conditional independent
    
    where, 
        D: set of attributes/data/columns that contains {D1, D2,...}
        h: a hypothesis, in classification, a class
        P(D|h) : Likelihood distribution of data given a specific hypothesis, in classification,
            likelihood distribution of data given a specific class
        P(h): Prior distribution
        P(h|D): Most likely hypothesis given data, in classification, most likely class given data
    """
    total_rows = len(dataset)
    summaries = summarize_by_class(dataset)
    prob = dict()
    max_prob, retClass = -inf, None
    for class_val, class_summaries in summaries.items():
        prob[class_val] = log(class_summaries[0][2] / float(total_rows))

        for i in range(len(class_summaries)):
            mean, stdev, _ = class_summaries[i]
            prob[class_val] += log_gausian(input_row[i], mean, stdev)
        if max_prob < prob[class_val]:
            retClass = class_val
            max_prob = prob[class_val]
    return retClass, prob

# Naive Bayes Algorithm
def naive_bayes(train_set, test_set):
    preidictions = list()
    for row in test_set:
        preidictions.append(MAP(train_set,row)[0])
        # print(MAP(train_set,row))
    return preidictions

## Work with Real Data

In [416]:
# Load a CSV file
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column])
        
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])  # concatenate lists of lists to a list
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            # test_set use to predict => no need to hold [class] data
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

## Gaussian Discriminnat Analaysis

Assumption:

- y ~ Bernoulli($\phi$) - 1 dimension
- x|y ~ Gaussian($\mu, \Sigma$) - d dimension

In [417]:
seed(1)
# Make a prediction with Naive Bayes on Iris Dataset
filename = 'data/sonar.csv'
filename = 'data/BankNote_Authentication.csv'
dataset = load_csv(filename)

# remove the string [attributes]
dataset.pop(0)

for i in range(len(dataset[0])-1): # except for the last column
	str_column_to_float(dataset, i)

# fit model
model = summarize_by_class(dataset)

# predict the label
n_folds = 5

scores = evaluate_algorithm(dataset, naive_bayes, n_folds)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [62.40875912408759, 63.868613138686136, 59.48905109489051, 66.42335766423358, 64.96350364963503]
Mean Accuracy: 63.431%


## Conclusion

This model cannot work with huge attributes, i.e `data/sonar.csv` due to the multiplication increase when # of attributes increase. The probability of each class rapidly converges to zero and we cannot use it to the the Maximum a Posteriori (MAP).

I tried to print the probability of these classes:

2.9139230184726384e-13

1.0403767988304062e-181

0.0

0.0

...

Problem: numercal unstability

Attemp: use log to avoid the numerical problem

```python
# prob[class_val] = (class_summaries[0][2] / float(total_rows))
prob[class_val] = log(class_summaries[0][2] / float(total_rows)
... 
# prob[class_val] *= (gauss(input_row[i], mean, stdev))
prob[class_val] += log_gausian(input_row[i], mean, stdev)
```

However, the average accuracy of this model on the `data\sonar.csv` dataset is just over 50% that is the same as the guessing.


In [418]:
X = np.array([[1,2,3,1],[1,2,2,1],[0,1,4,0]], np.float32)
y = np.array([x[-1] for x in X])
# print(X.shape, y.shape)
mean = [X[y==k].mean(axis = 0) for k in [0,1]]
mean

[array([0., 1., 4., 0.], dtype=float32),
 array([1. , 2. , 2.5, 1. ], dtype=float32)]

In [419]:
X_u = X.copy()

for k in [0,1]: X_u[y==k] -= mean[k]
stddev = X_u.T.dot(X_u) / len(y)
invStddev = np.linalg.pinv(stddev)
invStddev
len(mean)
# mean[1]
# X_u[y==1] -= mean[1]
X = [x[:-1] for x in X]

In [420]:
def fit(X, y):
    """
    Args:
        X: A list with (n x d); n datasize & [d] value (features)
        y: A list with (n x 1); n datasize & 1 value (label)
    Return:
        mean: (2 x d) (2 because we have 2 classes)
        covariance: (d x d) = 1/n * SUM
        fi: scalar (because we have 2 classes, fi = P(y = 1) = 1 - P(y = 0))
    """
    mean = np.array([X[y == k].mean(axis=0) for k in [0, 1]])
    X_u = X.copy()  # X_u: (n x d)
    for k in [0, 1]:
        X_u[y == k] -= mean[k]
        
    covariance = X_u.T.dot(X_u) / len(y) # X_u.T(d x n) * X_u(n x d) / n    -> (dxd)
    fi = y.mean()
    return mean, covariance, fi


def predict(X, u, invE, fi):
    """
        Args:
            X: (n x d)
            u: (2 x d)
            invE: (d x d)
            fi: scalar
        Return:
            Array of [n] predictions
    """
    return np.argmax([compute_prob(X, u[c], invE, fi, c) for c in [0, 1]], axis=0)


def compute_prob(X, u, invE, fi, c):
    """
    X: dataset, the last column is the ground truth
    c: class

    Return: P(X,c) = P(X|c)P(c)
        where,
        P(X|c) ~ Gaussian(u, E)
        P(c) ~ Berounlli(phi)
    """
    phi = fi**c * (1-fi)**(1-c)
    # return np.exp(-1.0 * np.sum((X - u).dot(invE)*(X - u), axis=1)) * phi    # sum up the row/features
    return -1.0* np.sum((X-u).dot(invE)*(X-u),axis=1) + log(phi) # sum up the row/features


def Binary_GDA(train_set, test_set):
    """
    Args:
        train_set: (n x (d+1)), with [d] features and 1 label column
        test_set: (n x (d+1)), with [d] features and 1 label column
    Return:
        A list of predictions: (1 x n)
    """ 
    y = np.array([x[-1] for x in train_set])
    X = np.array([x[:-1] for x in train_set])
    test_set = np.array([x[:-1] for x in test_set])

    u, E, fi = fit(X, y)
    invE = np.linalg.pinv(E)
    return predict(test_set, u, invE, fi)


In [421]:
X = np.array([[1, 2, 3, 1, 1],
              [1, 2, 2, 3, 1],
              [0, 1, 4, 1, 0],
              [1, 1, 1, 3, 1],
              [1, 2, 3, 4, 0]], np.float32)

y = np.array([x[-1] for x in X])
X = np.array([x[:-1] for x in X])

u, E, fi = fit(X, y)
invE = np.linalg.pinv(E)
phi0 = 1 - fi
phi1 = fi

# why element-wise product
L0 = np.exp(-1.0 * np.sum((X - u[0]).dot(invE)*(X-u[0]),axis=1) ) * phi0
# why element-wise product
L1 = np.exp(-1.0 * np.sum((X - u[1]).dot(invE)*(X-u[1]), axis=1) ) * phi1

print(np.argmax([L0, L1], axis=0))

[1 1 0 1 0]


In [422]:
seed(1)
# Make a prediction with Naive Bayes on Iris Dataset
filename = 'data/BankNote_Authentication.csv'
filename = 'data/sonar.csv'
dataset = load_csv(filename)

# remove the string [attributes]
dataset.pop(0)

for i in range(len(dataset[0])-1): # except for the last column
	str_column_to_float(dataset, i)

# Check if the last row is not INT{0,1}
if not isinstance(dataset[0][-1], int):
	print("The last row convert from str to int")
	str_column_to_int(dataset, len(dataset[0])-1)

# predict the label
n_folds = 5

scores = evaluate_algorithm(dataset, Binary_GDA, n_folds)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

The last row convert from str to int
Scores: [68.29268292682927, 75.60975609756098, 75.60975609756098, 75.60975609756098, 65.85365853658537]
Mean Accuracy: 72.195%


## Conclusion

I updated Binary_GDA(linear) and it's accuracy is increase to same as decision tree

## Refs
https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/